In [170]:
import numpy as np
import scipy as sp
from scipy import linalg

In [3]:
def generate_bernoulli_random_variable(probability):
    """乱数をベルヌーイ分布に従って発生する
    Args:
        probability: 成功確率
    Returns:
        0 or 1
    """
    return np.random.binomial(1, probability)

In [4]:
def generate_adjacent_value():
    """隣接行列の要素の値を生成する
    generate_adjacency_matrix() から呼ばれる
    論文の通り [-1.5, -0.5] か [0.5, 1.5] の値を生成する
    Returns:
        -1.5 ~ 0.5 か 0.5 ~ 1.5 の間の Float
    """
    signes = [1, -1]
    sign = np.random.choice(signes)
    return sign * (np.random.rand() + 0.5)

In [5]:
def generate_adjacency_matrix(shape):
    """下三角隣接行列を生成する
    Args:
        shape: 行列の形をタプルで受け取る。(3,4) なら 3 x 4 の行列を生成する
    Returns:
        下三角隣接行列
    """
    s = np.random.rand()
    B = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            if i > j:
                if generate_bernoulli_random_variable(s) != 0:
                    B[i][j] = generate_adjacent_value()
    return B

In [6]:
def generate_laplace_disturbance(size):
    """ラプラス分布に従うノイズ項を発生させる関数のファクトリ
    Returns:
        ラブラス分布に従うノイズ項の発生値
    """
    loc = 0
    scale = np.random.rand() * 2 + 1
    return np.random.laplace(loc, scale, size)

In [7]:
def generate_observed_data(B, e):
    """観測データを生成する
    Args:
        B: 隣接行列
        e: ノイズ項
    Returns:
        観測データを返す
    """
    X = np.zeros((p, n))
    X = e.copy()
    for i in range(B.shape[0]):
        for j in range(B.shape[1]):
            if B[i][j] != 0:
                X[i] += X[j]*B[i][j]
    return X            

In [8]:
def permuteta_order(X):
    """観測データの順番を入れ替える
    Args:
        X: 観測データ
    Returns:
        並び替えた観測データ
    """
    X = X.copy()
    for i in range(len(X)):
        a = np.random.randint(0, len(X) - 1)
        b = np.random.randint(0, len(X) - 1)
        tmp = X[a].copy()
        X[a] = X[b]
        X[b] = tmp
    return X

In [113]:
p = 5
n = 10
s = np.random.rand()
B = generate_adjacency_matrix((p, p))
e = generate_laplace_disturbance((p, n))
test_data = generate_observed_data(B, e)
test_data = permuteta_order(test_data)

In [219]:
import functools

In [10]:
def generate_gaussian_kernel(sigma):
    """ガウシアンカーネル生成する
    引数の分散を元にガウシアンカーネル関数を生成する
    カーネル関数は引数に値がペアで格納されたタプルを受け取りカーネル値を算出し返す
    Args:
        sigma: ガウシアン分布の分散
    Returns:
        ガウシアンカーネル関数        
    """
    return lambda x_i, x_j: np.exp((-1 / (2 * sigma)) * (x_i - x_j)**2)

In [11]:
def generate_gram_matrix(x, kernel_function):
    """グラム行列を生成する
    Args:
        x: 一次元配列
        kernel_function: カーネル関数
    Returns:
        x のサイズ n とすると、 n x n の グラム行列
    """
    return np.array([[kernel_function(xi, xj) for xj in x] for xi in x])    

In [25]:
def calc_MI_kernel_value(K_1, K_2, kappa=2*10**(-2)):
    """カーネル法を利用して相互情報量を計算する
    Args:
        K_1: 第一グラム行列
        K_2: 第二グラム行列
        kappa: 小さい正の値 論文では 2*10^(-2) だった
    Returns:
        相互情報量
    """
    n = len(K_1)
    numer_11 = (K_1 + ((n*kappa)/2)*np.eye(n, n))
    numer_11 = numer_11.dot(numer_11)
    numer_12 = K_1.dot(K_2)
    numer_21 = K_2.dot(K_1)
    numer_22 = (K_2 + ((n*kappa)/2)*np.eye(n, n))
    numer_22 = numer_22.dot(numer_22)
    numer = np.array([[numer_11, numer_12], [numer_21, numer_22]])
    denom_11 = numer_11.copy()
    denom_12 = np.zeros((n, n))
    denom_21 = np.zeros((n, n))
    denom_22 = numer_22.copy()
    denom = np.array([[denom_11, denom_12], [denom_21, denom_22]])
    # -1/2*log(det(numer)/det(denom))
    # ""行列の行列"の行列式？
    # なにかしらのスカラー値が返るはずのでとりあえず乱数返す
    return np.random.randint(100)

In [73]:
def calc_residual(x_j, x_i):
    """残差を計算する
    Args:
        x_j: 一次元配列 x_i の成分を取り除きたい
        x_i: 一次元配列 この成分を x_j から取り除きたい
    Returns:
        x_j から x_i の成分を取り除いた一次元配列
    """
    return x_j- ((np.cov(x_j, x_i, bias=1)[0][1] / np.var(x_j)) * x_j)

In [162]:
def calc_T_kernel_value(X, j):
    """j 以外の 添字 i について X[i] と残差の相互情報量をの総和を計算する
    R_j は R_j[i] で X[j] から X[i] の成分を取り除いた残差を得られる
    Args:
        X: 観測データ
        j: 現在注目している観測データの添字
    Returns:
        X[j] の独立度を測る指標
    """
    R_j = np.array([calc_residual(X[j], X[i]) for i in X.keys() if i != j])
    return sum([calc_MI_kernel_value(X[j], R_j_i) for R_j_i in R_j])    

In [166]:
def find_most_independent_variable(X):
    """最も独立な変数を見つける
    観測データのそれぞれにおいて、独立度を calc_T_kernel_value() で計算し
    sort して最も小さな値を持つ変数の添字を返す
    Args:
        X: 観測データ
    Returns:
        最も独立な変数の添字
    """
    return sorted({j: calc_T_kernel_value(X, j) for j in X.keys()}.items(), key=lambda x: x[1])[0][0]    

In [169]:
def DirectLiNGAM(X):
    """因果関係を推論する
    Args:
        X: 観測データ
    Returns:
        因果関係の順番の一次元配列
    Todo:
        隣接行列を返すようにする
    """
    K = []
    X = {i:test_data[i] for i in range(len(test_data))}
    while (len(X) > 0):
        k = find_most_independent_variable(X)
        K.append(k)
        X = {i:calc_residual(X[k], X[i]) for i in X.keys() if i != k}
    return K

[2, 0, 1, 3, 4]